---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [ ]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [24]:
import pandas as pd
import numpy as np
import re

def cleanup(row):
    m1 = re.search(r"(\w+)\[edit\]",row[0])
    if m1 is not None:
        row['State'] = str(m1.group(1)).strip()
        row['RegionName'] = np.nan
    else:
        row['State'] = ''
        row['RegionName'] = str(re.sub(r"\(.*$",'',row[0])).strip()
    return row

def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    univtowns_df = (pd.read_csv('university_towns.txt', sep='\n', header=None, names=[0,'State','RegionName']).
                   apply(cleanup, axis=1))
    state=''
    for iter in range(univtowns_df.shape[0]):
        if univtowns_df.iloc[iter]['State'] == '':
            univtowns_df.iloc[iter]['State'] = state
        elif univtowns_df.iloc[iter]['State'] != '':
            state = univtowns_df.iloc[iter]['State']

    return (univtowns_df.dropna().drop(labels=[0],axis=1))


get_list_of_university_towns()

,State,RegionName
1,Alabama,Auburn
2,Alabama,Florence
3,Alabama,Jacksonville
4,Alabama,Livingston
5,Alabama,Montevallo
6,Alabama,Troy
7,Alabama,Tuscaloosa
8,Alabama,Tuskegee
10,Alaska,Fairbanks
12,Arizona,Flagstaff


In [65]:
import pandas as pd
import numpy as np

def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    gdp_df = (pd.read_excel('gdplev.xls', header=5, usecols=[4,5,6], index_col=0)['2000q1':])
    gdp_df = (gdp_df.reset_index()
              .dropna())
    
    gdp_df['Category']=''
    
    for idx in gdp_df.index.unique():
        if (idx-1)>=0:
            if gdp_df['GDP in billions of chained 2009 dollars'].iloc[idx] < gdp_df['GDP in billions of chained 2009 dollars'].iloc[idx-1]:
                #gdp_df['Category'].iloc[idx]='Decline'
                gdp_df.loc[idx,'Category']='Decline'
                
                if gdp_df['Category'].iloc[idx-1] == 'Growth' and \
                    gdp_df['GDP in billions of chained 2009 dollars'].iloc[idx] > gdp_df['GDP in billions of chained 2009 dollars'].iloc[idx+1]:
                    #gdp_df['Category'].iloc[idx]='Recession Start'
                    gdp_df.loc[idx,'Category']='Recession Start'
                
            elif gdp_df['GDP in billions of chained 2009 dollars'].iloc[idx] > gdp_df['GDP in billions of chained 2009 dollars'].iloc[idx-1]:
                #gdp_df['Category'].iloc[idx]='Growth'
                gdp_df.loc[idx,'Category']='Growth'
    
    return str(gdp_df[gdp_df['Category']=='Recession Start'].reset_index().iloc[0]['index'])
    
get_recession_start()

'2008q3'

In [69]:
import pandas as pd
import numpy as np

def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    quarterofrecessionstart = get_recession_start()
    
    gdp_df = pd.read_excel('gdplev.xls', header=5, usecols=[4,5,6], index_col=0)[quarterofrecessionstart:]
    gdp_df = gdp_df.reset_index().dropna()
    
    
    gdp_df['Category']=''
    idxstart = gdp_df.first_valid_index()
    gdp_df.loc[idxstart,'Category'] = 'Recession Start'
    
    for idx in gdp_df.index.unique():
        if (idx-1)>=0:
            if gdp_df['GDP in billions of chained 2009 dollars'].iloc[idx] < gdp_df['GDP in billions of chained 2009 dollars'].iloc[idx-1]:
                #gdp_df['Category'].iloc[idx]='Decline'
                gdp_df.loc[idx, 'Category'] = 'Decline'
                
            elif gdp_df['GDP in billions of chained 2009 dollars'].iloc[idx] > gdp_df['GDP in billions of chained 2009 dollars'].iloc[idx-1]:
                #gdp_df['Category'].iloc[idx]='Growth'
                gdp_df.loc[idx, 'Category']='Growth'
                
                if gdp_df['Category'].iloc[idx-1] == 'Decline' and \
                    gdp_df['Category'].iloc[idx-2] == 'Decline' and \
                    gdp_df['GDP in billions of chained 2009 dollars'].iloc[idx+1] > gdp_df['GDP in billions of chained 2009 dollars'].iloc[idx]:
                    #gdp_df['Category'].iloc[idx-1]='Recession End'
                    gdp_df.loc[idx-1, 'Category']='Recession End'
    
       
    return str(gdp_df[gdp_df['Category']=='Recession End'].reset_index().iloc[0]['index'])
get_recession_end()

'2009q2'

In [70]:
import pandas as pd
import numpy as np

def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    quarterofrecessionstart = get_recession_start()
    quarterofrecessionend = get_recession_end()
    
    gdp_df = (pd.read_excel('gdplev.xls', header=5, usecols=[4,5,6], index_col=0)[quarterofrecessionstart:quarterofrecessionend])
    
    return str(gdp_df['GDP in billions of chained 2009 dollars'].idxmin())

get_recession_bottom()

'2009q2'

In [83]:
import pandas as pd
import numpy as np
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    yearsofinterest=['2000']
    
    house_df = pd.read_csv('City_Zhvi_AllHomes.csv', header=0, na_values='')
    house_df['2000q1']=np.nan
    
    for year in yearsofinterest:
        house_df['2000q1'] = house_df.groupby(level=0)[['%s-01'%year,'%s-02'%year,'%s-03'%year]].agg(np.mean())
    
    #for year in yearsofinterest: 
    #   house_df['%sq1'%year] = house_df['%s-01'%(year)] + house_df['%s-02'%(year)] + house_df['%s-01'%(year)]

    return house_df

convert_housing_data_to_quarters()

TypeError: mean() missing 1 required positional argument: 'a'

In [59]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    return "ANSWER"